# Extract the neiborhood data of Toronto from wikipedia

### Last week, we learnt download dataset from FourSqure API. This  time, let's scrap the information from Wikipedia and make them grouped

First, let's import the libary we need

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
print('libraries imported successfully')

libraries imported successfully


Now, we are going to use beautiful soup to scrap the data from url

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
# make soup
soup = BeautifulSoup(page,'html.parser')
tables = soup.select('table')
table = tables[0].findAll('tr')

In [3]:
postcodes = [] #creat list to contain the data
broughs = []
neighborhoods = []
for row in tables[0].findAll('tr'): # find each row
    if row.findAll('td'):           # the data start with td
        postcodes.append(row.findAll('td')[0].contents[0]) # the first column is postcode.
        broughs.append(row.findAll('td')[1].contents[0].string.replace('\n',''))# the second column is broughs. delete the '\n' if ther is
        neighborhoods.append(row.findAll('td')[2].contents[0].string.replace('\n',''))# the third column is neighborhood.delete the '\n' if there is
# convert the data into dataframe
df = pd.DataFrame({'Postcode':postcodes, 'Broughs':broughs, 'Neighborhoods':neighborhoods})

#### let's clean the data further

In [4]:
df = df[df.Broughs!='Not assigned'] # let's delete those brough not assigned
nh_mask = df.Neighborhoods =='Not assigned' # where neighbor hood was not assigned
df["Neighborhoods"][nh_mask] = df['Broughs'][nh_mask]
df.shape

(211, 3)

In [57]:
df_group = df.groupby('Postcode').apply(f)

In [60]:
df.head()

,Postcode,Broughs,Neighborhoods
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


We can find the row 2 and row 3 has the same Postocode and same brough with different neigborhood. We are suposed to merge them. So, we are going to use groupby()

In [56]:
# to define a function applied to groupby, here I acknowlwdge the bbs https://codeday.me/bug/20171205/104918.html
#It told me how to merge string if we jsut want to join them together
def f(x):
     return pd.DataFrame(dict(Postcode = np.unique(x['Postcode']),  #We extract the unique postcodes
                        Broughs = np.unique(x['Broughs']),          #We extract the unique broughs, since many are the same with the same postcodes
                        Neighborhoods = "%s" % ', '.join(x['Neighborhoods']))) # merge the neighborhood

In [61]:
df_group.shape

(103, 3)

In [63]:
df_group.reset_index(drop = True, inplace = True) # let's drop the index

Let's print the first 5 postcodes and neighborhoods

In [64]:
df_group.head()

,Postcode,Broughs,Neighborhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now, we know there is 103 postcodes if we delete the unasigned broughs and neighborhoods